In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [1]:
import tifffile
import numpy as np
import os
import cv2

from skimage.exposure import rescale_intensity
from stardist.models import StarDist2D

from src.utils.helpers import cut_out_image

from src.utils.mask_creation import MaskCreator

ModuleNotFoundError: No module named 'src'

In [2]:
image_paths = sorted(
    [
        "data/layer_examples/" + path
        for path in os.listdir("data/layer_examples")
        if not path.startswith(".")
    ]
)
images = list(map(tifffile.imread, image_paths))

In [3]:
model_path = "models/decision_tree_model.joblib"
mask_creator = MaskCreator(model_path)

|2025-11-05|14:03:39.876| [WARNING] base.py (376): Trying to unpickle estimator DecisionTreeClassifier from version 1.7.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

|2025-11-05|14:03:40.177| [WARNING] base.py (376): Trying to unpickle estimator RandomForestClassifier from version 1.7.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



In [5]:
# for image in images:
#     mask = mask_creator.create_mask(image)
#     compare_two_images(image, mask)

In [12]:
test_image, test_mask = data[0]

In [4]:
model = StarDist2D.from_pretrained("2D_versatile_he")

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


In [13]:
test_image_norm = rescale_intensity(test_image, out_range=(0, 1))
test_labels, test_dict = model.predict_instances(
    test_image_norm, axes="YXC", prob_thresh=0.05, nms_thresh=0.3, return_labels=True
)
test_labels = cut_out_image(test_labels, test_mask)

In [15]:
nuclei_mask = np.uint8(test_labels)

In [6]:
# # Create distance map
# distance_map = cv2.distanceTransform(nuclei_mask, cv2.DIST_L2, 5)
# surface = -distance_map
# compare_two_images(surface, distance_map)

In [7]:
# # Define markers
# ret, markers = cv2.connectedComponents(nuclei_mask)
# markers = markers.astype(np.int32)
# plt.imshow(markers)
# plt.axis(False)
# plt.show()

In [ ]:
dummy_img = np.zeros((*nuclei_mask.shape, 3), dtype=np.uint8)
labeled_test_img = cv2.watershed(dummy_img, markers)

In [8]:
# compare_two_images(test_image, labeled_test_img)